In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
import json

/opt/conda/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='/data/ephemeral/home/dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'val_split_random.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2024
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash' # 평가 모델 위치

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [ ]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/faster_rcnn_r50_fpn_1x_val_trash/latest.pth


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 974/974, 34.2 task/s, elapsed: 28s, ETA:     0s

In [7]:
# submission 양식에 맞게 output 후처리
results = []
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    img_id = img_ids[i]
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=img_id))[0]
    for j in range(class_num):
        for o in out[j]:
            
            result_dict = {
                "image_id": image_info['id'],
                "category_id": j,
                "bbox": [float(o[0]), float(o[1]), float(o[2]), float(o[3])],
                "score": float(o[4])
            }
            results.append(result_dict)

            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

# json 파일
json_file = os.path.join(cfg.work_dir, 'val_prediction' + f'_{epoch}.json')
with open(json_file, 'w') as f:
    json.dump(results, f)

# csv 파일
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, 'val_prediction' + f'_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


,PredictionString,image_id
0,0 0.9863251 736.64453 32.613983 949.3738 448.3...,train/0003.jpg
1,0 0.60986423 332.30084 276.40402 757.8 855.204...,train/0006.jpg
2,0 0.57219845 447.03336 450.61316 544.6425 579....,train/0011.jpg
3,0 0.16024078 256.05768 358.36267 386.52167 389...,train/0019.jpg
4,0 0.9719902 391.86243 307.79303 794.79553 751....,train/0024.jpg
